## NLP Tutorial

NLP - or *Natural Language Processing* - is shorthand for a wide array of techniques designed to help machines learn from text. Natural Language Processing powers everything from chatbots to search engines, and is used in diverse tasks like sentiment analysis and machine translation.

In this tutorial we'll look at this competition's dataset, use a simple technique to process it, build a machine learning model, and submit predictions for a score!

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

### A quick look at our data

Let's look at our data... first, an example of what is NOT a disaster tweet.

In [4]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

And one that is:

In [5]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

### Building vectors

The theory behind the model we'll build in this notebook is pretty simple: the words contained in each tweet are a good indicator of whether they're about a real disaster or not (this is not entirely correct, but it's a great place to start).

We'll use scikit-learn's `CountVectorizer` to count the words in each tweet and turn them into data our machine learning model can process.

Note: a `vector` is, in this context, a set of numbers that a machine learning model can work with. We'll look at one in just a second.

In [6]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [7]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


The above tells us that:
1. There are 54 unique words (or "tokens") in the first five tweets.
2. The first tweet contains only some of those unique tokens - all of the non-zero counts above are the tokens that DO exist in the first tweet.

Now let's create vectors for all of our tweets.

In [8]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

### Our model

As we mentioned above, we think the words contained in each tweet are a good indicator of whether they're about a real disaster or not. The presence of particular word (or set of words) in a tweet might link directly to whether or not that tweet is real.

What we're assuming here is a _linear_ connection. So let's build a linear model and see!

In [9]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Let's test our model and see how well it does on the training data. For this we'll use `cross-validation` - where we train on a portion of the known data, then validate it with the rest. If we do this several times (with different portions) we can get a good idea for how a particular model or method performs.

The metric for this competition is F1, so let's use that here.

In [10]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.60387232, 0.57511962, 0.64485082])

The above scores aren't terrible! It looks like our assumption will score roughly 0.65 on the leaderboard. There are lots of ways to potentially improve on this (TFIDF, LSA, LSTM / RNNs, the list is long!) - give any of them a shot!

In the meantime, let's do predictions on our training set and build a submission for the competition.

In [11]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [12]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [13]:
sample_submission["target"] = clf.predict(test_vectors)

In [14]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [15]:
sample_submission.to_csv("submission.csv", index=False)

In [16]:
print(train_df.columns)    
print('---')
if 0 in train_df['target']:
    print('true')
    
for i in range(0,len(train_df['target'])):
    if train_df['target'][i] == 0:
        print(f'index: {i} is 0')
    if i is 20:
        break
print('---')        
for i in range(0, len(train_df['location'])):
    if isinstance(train_df['location'][i], (list, str)):
        print(train_df['location'][i])
    if i is 40:
        break
print('---')
size=len(train_df['id'])

count_with_location=0
count_without_location=0
for i in range(0, size):
    if isinstance(train_df['location'][i], (list, str)) == 1 and train_df['target'][i] == 1:
        #print(train_df['id'][i])
        count_with_location+=1
    elif isinstance(train_df['location'][i], (list, str)) == 0 and train_df['target'][i] == 1:
        count_without_location+=1
print(count_with_location) # 2196
print(count_without_location) # 1075\
print('---')
count_with_keyword=0
count_without_keyword=0
for i in range(0, size):
    if train_df['target'][i]==1 and isinstance(train_df['keyword'][i],(list,str))==1:
        count_with_keyword+=1
    elif train_df['target'][i]==1 and isinstance(train_df['keyword'][i],(list,str))==0:
        count_without_keyword+=1
print(count_with_keyword, count_without_keyword) # 3229, 42

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
---
true
index: 15 is 0
index: 16 is 0
index: 17 is 0
index: 18 is 0
index: 19 is 0
index: 20 is 0
---
Birmingham
Est. September 2012 - Bristol
AFRICA
Philadelphia, PA
London, UK
Pretoria
World Wide!!
Paranaque City
Live On Webcam
---
2196
1075
---
3229 42


In [19]:
correlation_matrix=train_df.corr(method='spearman')

In [20]:
print(correlation_matrix)

              id    target
id      1.000000  0.060789
target  0.060789  1.000000


In [24]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df = pd.DataFrame(train_df)

In [25]:
df['keyword'] = df['keyword'].fillna('missing').astype(str)

In [28]:
df['keyword_encoded'] = label_encoder.fit_transform(df['keyword'])

correlation_matrix = df[['target', 'keyword_encoded']].corr(method='pearson')
print(correlation_matrix)

                   target  keyword_encoded
target           1.000000         0.057669
keyword_encoded  0.057669         1.000000


Now, in the viewer, you can submit the above file to the competition! Good luck!